# 11 PLOTTING AND MORE ABOUT CLASSES

## 11.2 Plotting Mortgages, an Extended Example

In Chapter 8, we worked our way through a hierarchy of mortgages as way of illustrating the use of subclassing. We concluded that chapter by observing that 

#### “our program should be producing plots designed to show how the mortgage behaves over time.” 

Figure 11.1 enhances class Mortgage by adding methods that make it convenient to produce such plots.

In [27]:
#Following function is defined in Chapter 8, and used here
import numpy as np

def findPayment(loan, r, m):
    """Assumes: loan and r are floats, m an int
       Returns the monthly payment for a mortgage of size
       loan at a monthly rate of r for m months"""
    return loan*((r*(1+r)**m)/((1+r)**m - 1))

class Mortgage(object):
    """Abstract class for building different kinds of mortgages"""
    
    def __init__(self, loan, annRate, months):
        """Create a new mortgage"""
        self.loan = loan
        self.rate = annRate/12.0
        self.months = months
        self.paid = [0.0] 
        self.owed = [loan]
        self.payment = findPayment(loan, self.rate, months)
        self.legend = None #description of mortgage
        
    def makePayment(self):
        """Make a payment"""
        self.paid.append(self.payment)
        reduction = self.payment - self.owed[-1]*self.rate
        self.owed.append(self.owed[-1] - reduction)
        
    def getTotalPaid(self):
        """Return the total amount paid so far"""
        return sum(self.paid)
    
    def __str__(self):
        return self.legend

    def plotPayments(self, style):
        plt.plot(self.paid[1:], style, label = self.legend)
    
    def plotBalance(self, style):
        plt.plot(self.owed, style, label = self.legend)
        
    def plotTotPd(self, style):
        """Plot the cumulative total of the payments made"""
        totPd = [self.paid[0]]
        for i in range(1, len(self.paid)):
            totPd.append(totPd[-1] + self.paid[i])
        plt.plot(totPd, style, label = self.legend)
    
    def plotNet(self, style):
        """Plot an approximation to the total cost of the mortgage
           over time by plotting the cash expended minus the equity
           acquired by paying off part of the loan"""
        totPd = [self.paid[0]]
        for i in range(1, len(self.paid)):
            totPd.append(totPd[-1] + self.paid[i])
       
    #Equity acquired through payments is amount of original loan
        #  paid to date, which is amount of loan minus what is still owed
        equityAcquired = np.array([self.loan]*len(self.owed))
        equityAcquired = equityAcquired - np.array(self.owed)
        
        net =np.array(totPd) - equityAcquired
        
        plt.plot(net, style, label = self.legend)


The methods `plotPayments` and `plotBalance` are simple one-liners, but they do use a form of `pylab.plot` that we have not yet seen.

The nontrivial methods in class Mortgage are `plotTotPd` and `plotNet`. 

The method `plotTotPd` simply plots the cumulative total of the payments made. 

The method `plotNet` plots an approximation to the total cost of the mortgage over time by plotting the cash expended minus the equity acquired by paying off part of the loan

####  List to Numpy's array

The expression `np.array(self.owed)` in plotNet performs a type conversion :list to numpy array

There are a number of `convenient` ways to `manipulate arrays` that are not readily available for lists.

There are a number of ways to create `arrays` in `Numpy`, but the most common way is:*first create a list, and then convert it*.


In [28]:
import numpy as np
a1 = np.array([1, 2, 4])
print('a1 =', a1)
a2 = a1*2
print('a2 =', a2)
print('a1 + 3 =', a1 + 3)
print('3 - a1 =', 3 - a1)
print('a1 - a2 =', a1 - a2)
print('a1*a2 =', a1*a2)

a1 = [1 2 4]
a2 = [2 4 8]
a1 + 3 = [4 5 7]
3 - a1 = [ 2  1 -1]
a1 - a2 = [-1 -2 -4]
a1*a2 = [ 2  8 32]


the next code repeats the three subclasses of Mortgage from Chapter 8.

In [ ]:

class Fixed(Mortgage):
    def __init__(self, loan, r, months):
        Mortgage.__init__(self, loan, r, months)
        self.legend = 'Fixed, ' + str(r*100) + '%' 

class FixedWithPts(Mortgage):
    def __init__(self, loan, r, months, pts):
        Mortgage.__init__(self, loan, r, months)
        self.pts = pts
        self.paid = [loan*(pts/100.0)]
        self.legend = 'Fixed, ' + str(r*100) + '%, '\
                   + str(pts) + ' points'

class TwoRate(Mortgage):
    def __init__(self, loan, r, months, teaserRate, teaserMonths):
        Mortgage.__init__(self, loan, teaserRate, months)
        self.teaserMonths = teaserMonths
        self.teaserRate = teaserRate
        self.nextRate = r/12.0
        self.legend = str(teaserRate*100)\
                      + '% for ' + str(self.teaserMonths)\
                      + ' months, then ' + str(r*100) + '%'

    def makePayment(self):
        if len(self.paid) == self.teaserMonths + 1:
            self.rate = self.nextRate
            self.payment = findPayment(self.owed[-1], self.rate,
                                       self.months - self.teaserMonths)
        Mortgage.makePayment(self)


The next code contain functions that can be used to generate plots intended to provide insight about the different kinds of mortgages.

In [ ]:
def plotMortgages(morts, amt):
    styles = ['b-', 'b-.', 'b:']
   
    #Give names to figure numbers
    payments = 0
    cost = 1
    balance = 2
    netCost = 3
    # titles and axis labels for each plot 
    plt.figure(payments)
    plt.title('Monthly Payments of Different $' + str(amt)
                + ' Mortgages')
    plt.xlabel('Months')
    plt.ylabel('Monthly Payments')
    
    plt.figure(cost)
    plt.title('Cash Outlay of Different $' + str(amt) + ' Mortgages')
    plt.xlabel('Months')
    plt.ylabel('Total Payments')
    
    plt.figure(balance)
    plt.title('Balance Remaining of $' + str(amt) + ' Mortgages')
    plt.xlabel('Months')
    plt.ylabel('Remaining Loan Balance of $')
    
    plt.figure(netCost)
    plt.title('Net Cost of $' + str(amt) + ' Mortgages')
    plt.xlabel('Months')
    plt.ylabel('Payments - Equity $')
    
    # produce the actual pl
    for i in range(len(morts)):
        plt.figure(payments)
        morts[i].plotPayments(styles[i])
        
        plt.figure(cost)
        morts[i].plotTotPd(styles[i])
        
        plt.figure(balance)
        morts[i].plotBalance(styles[i])
        
        plt.figure(netCost)
        morts[i].plotNet(styles[i])
   
    # legend
    plt.figure(payments)
    plt.legend(loc = 'upper center')
    
    plt.figure(cost)
    plt.legend(loc = 'best')
    
    plt.figure(balance)
    plt.legend(loc = 'best') 

def compareMortgages(amt, years, fixedRate, pts, ptsRate,
                    varRate1, varRate2, varMonths):
    totMonths = years*12
    fixed1 = Fixed(amt, fixedRate, totMonths)
    fixed2 = FixedWithPts(amt, ptsRate, totMonths, pts)
    twoRate = TwoRate(amt, varRate2, totMonths, varRate1, varMonths)
    morts = [fixed1, fixed2, twoRate]
    for m in range(totMonths):
        for mort in morts:
            mort.makePayment()
    
    
    plotMortgages(morts, amt)

In [ ]:
compareMortgages(amt=200000, years=30, fixedRate=0.07,
                 pts = 3.25, ptsRate=0.05,
                 varRate1=0.045, varRate2=0.095, varMonths=48)

The function plotMortgages generates appropriate <b>titles and axis labels</b> for each plot, 

and then uses the methods in MortgagePlots to <b>produce the actual plots</b>.

* 1 `plotPayments` : the first plot makes it clear <b>how the monthly payments vary (or don’t) over time</b>


* 2 `plotTotPd` : It sheds some light on the cost of each kind of mortgage by plotting the <b>cumulative costs</b> that have been incurred at the start of each month.


* 3 `plotBlance`: show the remaining debt


* 4 `plotNet`: total net cost of having the mortgage


## Fuether Reading

* **Matplotlib Tutorials:** https://matplotlib.org/tutorials/index.html
